# Homework 3
Justin Wagers <br>
Maryam Blooki <br>
Sunny Vidhani <br>
Daniel Stern <br>
Kaiwen Zhang <br>

## Instagram Scaper


In [ ]:
!pip install instaloader
import instaloader
import time
import pandas as pd
import sys
sys.setrecursionlimit(10000)
from datetime import datetime
from itertools import dropwhile,takewhile
!pip install --user google-cloud

Scraping 400 posts from the Zara page (we got blocked by Instagram after 220 so that's the size of our resulting dataset). 

In [ ]:
L = instaloader.Instaloader()
L.login('text.2395', 'text.2395!')
df = pd.DataFrame()
from pprint import pprint

i =0
n = 0
for post in instaloader.Profile.from_username(L.context, 'zara').get_posts():
  # print(post.comments)
  if post.typename == 'GraphImage':
    df = df.append({'Caption': post.caption,'Comment_Count': post.comments, 'Type': post.typename, 'Likes': post.likes, 'URL': post.url}, ignore_index = True)
    df.to_excel("insta_data.xlsx", index = False)
    n = n+1
  i = i+1
  if n >400:
    break


## Google Image Analytics


In [ ]:
!pip install --user google-cloud
!pip install --user google-cloud-vision
import xlrd
from google.cloud import vision
import os
import pandas as pd


Running the scraped Instagram url's through Google Vision to receive image labels. 

In [ ]:
Application_Credentials = 'Text Analytics Image Analytics-7c7508f3a4e3.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.Image()

loc = ('insta_data (8).xlsx')
wb =xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0,0)
df = pd.DataFrame()

for i in range(sheet.nrows):
  image_src_temp = sheet.cell_value(i,4)
  image.source.image_uri = image_src_temp
  response = client.label_detection(image = image)
  labels = response.label_annotations
  l = []
  for label in labels:
    l.append(label.description)
  s= ' '.join(l)
  print("s")
  print(s)
  df = df.append({'URL': image_src_temp, 'Labels': s}, ignore_index = True)

df.to_excel("Labels.xlsx", index = False)


In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

Re-importing data 

In [ ]:
import pandas as pd

insta_data = pd.read_excel(r"Instagram Image Data.xlsx")

## Task A

Here we create a weighted engagement score after normalizing likes and comments with a 40% weight on likes, 60% weight on comments. 

We then create an 'engagement level' binary bin with above the median engagement scores classified as a "1" and below the median engagement scores classified as a "2". 

In [ ]:
insta_data['Likes_Norm'] = insta_data['Likes'] / insta_data['Likes'].sum()
insta_data['Comment_Count_Norm'] = insta_data['Comment_Count'] / insta_data['Comment_Count'].sum()

In [ ]:
insta_data['Engagement Score'] = 0.4 * insta_data['Likes_Norm'] + 0.6 * insta_data['Comment_Count_Norm']
insta_data['Engagement Level'] = ''

for i in range(len(insta_data)):
  if insta_data.loc[i,'Engagement Score'] > insta_data['Engagement Score'].median():
    insta_data.loc[i,'Engagement Level'] = 1
  else:
    insta_data.loc[i,'Engagement Level'] = 0

In [ ]:
insta_data

,Caption,Comment_Count,Likes,Type,URL,Labels,Likes_Norm,Comment_Count_Norm,Engagement Score,Engagement Level
0,FW20 campaign. Coming soon #zaracampaign #zara...,260,35740,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Cool Human Font Fur Outerwear Textile Album co...,0.002328,0.003285,0.002902,0
1,FW20 campaign. Coming soon #zaracampaign #zara...,190,38231,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Clothing Formal wear Suit Standing Coat Black-...,0.002490,0.002401,0.002437,0
2,FW20 campaign. Coming soon #zaracampaign #zara...,113,29261,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Suit Fashion Formal wear Outerwear Fashion design,0.001906,0.001428,0.001619,0
3,Archive Collection 1996/2012 — Launching a re-...,192,59000,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Clothing Black Standing Outerwear Coat Fashion...,0.003843,0.002426,0.002993,0
4,Archive Collection 1996/2012 — Launching a re-...,152,46610,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Face Hair Eyebrow Hairstyle Forehead Chin Skin...,0.003036,0.001921,0.002367,0
...,...,...,...,...,...,...,...,...,...,...
208,Zara Man and the Estate of Eduardo Chillida co...,73,41553,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Photograph Sitting Photography Black-and-white...,0.002706,0.000922,0.001636,0
209,AWAKENING | Fine Knitwear collection. Key tran...,342,113907,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Sitting Beauty Botany Fashion Tree Photography...,0.007419,0.004322,0.005560,1
210,AWAKENING | Fine Knitwear collection. Key tran...,195,56143,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,White Clothing Pink Shoulder Sitting Fashion D...,0.003657,0.002464,0.002941,0
211,Boots evolve into modern looks with heavy lug ...,92,46018,GraphImage,https://scontent-hkt1-1.cdninstagram.com/v/t51...,Leg Sitting Leather Fashion Footwear Thigh Gam...,0.002997,0.001163,0.001896,0


## Task B


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
df = pd.read_excel ('Instagram Image Data.xlsx')
insta = pd.read_excel ('Instagram Image Data.xlsx')
vectorizer = TfidfVectorizer()


Designating the engagement level as the dependent variable.


In [ ]:

df['engagement'] = insta_data['Engagement Level']


In [ ]:
y = df['engagement'].values.astype('int')

Creating the tf-idf matrix for the image labels as the independent variables


In [ ]:
from scipy import sparse
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
label_list = list(df['Labels'].values)

sparse_matrix = tfidf_vectorizer.fit_transform(label_list)
doc_term_matrix = sparse_matrix.todense()

label_vector = pd.DataFrame(doc_term_matrix, 
                    columns=tfidf_vectorizer.get_feature_names())

label_vector

,abdomen,accessory,adaptation,afro,airsoft,akita,album,alcoholic,ankle,aqua,architecture,area,arm,army,arranging,art,athletic,atmosphere,azure,baby,bar,barechested,barefoot,bathing,bathroom,bathtub,beach,beanie,beauty,bed,bedding,beige,beverage,bicycle,bikini,black,blazer,blond,blouse,blue,...,text,textile,thigh,tights,tile,tourism,traffic,transport,travel,tree,trench,trial,trousers,trunk,turquoise,tuxedo,undergarment,uniform,urban,vacation,vehicle,vertebrate,vintage,waist,wall,water,wax,wear,wheel,white,wig,window,wood,woodland,wool,woolen,worker,world,yellow,youth
0,0.0,0.320067,0.0,0.0,0.0,0.0,0.354441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.261157,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266584,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.334310,0.0,0.214135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420291,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.261546,0.357266,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327992,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209229,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.321754,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.308296,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.247640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.214419,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.432448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.232321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0

Fitting a logistic regression model to predict engagement with the image labels. 

In [ ]:
label_logr = LogisticRegression(random_state=0).fit(label_vector,y)
label_pred = label_logr.predict(label_vector)
label_score = label_logr.score(label_vector,y)
label_score


0.8450704225352113

In [ ]:
from sklearn.metrics import confusion_matrix
label_matrix = confusion_matrix(label_pred,y)
print(label_matrix)

[[90 16]
 [17 90]]


We can see that the image labels give a model accuracy of about 85%, with an approximately equal number of false positives and false negatives. 

## Use Caption Text


Repeating the model fitting process, this time using the caption text instead of the google vision image labels. 

In [ ]:
import string
rem = string.punctuation
pattern = r"[{}]".format(rem)
df['clean_caption'] = df['Caption'].str.replace(pattern, ' ')

caption_list = list(df['clean_caption'].values)

sparse_matrix = tfidf_vectorizer.fit_transform(caption_list)
caption_matrix = sparse_matrix.todense()


caption_vector = pd.DataFrame(caption_matrix, 
                    columns=tfidf_vectorizer.get_feature_names())

caption_vector



,004,005,04,1903,1970,1996,1998,2003,2011,2012,2020,24k,3147,70,90,97,99,accessories,acetate,active,add,alfonso,alicemoireau,allison,allure,amber,amsterdam,andy,animal,anja,anna,approach,archive,archives,aromatic,arsun,art,artist,arts,assertive,...,vibes,vibrant,view,vincent,vinyl,virgin,visual,volumes,waistband,wardrobe,warhol,warholfoundation,waste,water,wear,welcome,wide,winter,woman,wool,work,working,wrap,z1975,zara,zaraaarchives,zaraarchive,zaraarchives,zaracampaign,zaracandles,zaracities,zaraedit,zaraedited,zarakids,zaraman,zaramini,zaranewin,zaraviews,zarawoman,zimmermann
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.402054,0.0,0.0,0.0,0.0,0.495759,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.458611,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.136595,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.412862,0.0,0.0,0.0,0.0,0.000000,0.452418,0.0,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.167856,0.000000,0.0,0.215431,0.167856,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.167856,0.167856,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.111345,0.0,0.0,0.189108,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.056325,0.0
4,0.0,0.0,0.0,0.0,0.0,0.172735,0.194605,0.0,0.000000,0.172735,0.0,0.0,0.0,0.0,0.0,0.0,0.194605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.194605,0.0,0.0,0.0,0.0,0.0,0.0,0.172735,0.172735,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.178759,0.0,0.0,0.000000,0.0,0.0,0.0,0.114582,0.0,0.0,0.194605,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.057962,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.226865,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.204864,0.0,0.0,0.0,0.126281,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.117512,0.0,0.000000,0.0,0.000000,0.0
209,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.105024,0.0
210,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000

In [ ]:
caption_logr = LogisticRegression(random_state=0).fit(caption_vector,y)
caption_pred = caption_logr.predict(caption_vector)
caption_score = caption_logr.score(caption_vector,y)
caption_score

0.8591549295774648

In [ ]:
caption_matrix = confusion_matrix(caption_pred,y)
print(caption_matrix)


[[91 14]
 [16 92]]


We can see that running a model using only the caption text gives a similar model accuracy of about 86%, again with an approximately equal number of false positives and false negatives. 


## Use Label and Caption


Repeating the model fitting process, this time using tf-idf information for poth post caption text and google vision image labels to compare accuracy. 

In [ ]:
combine_vector = pd.concat([label_vector, caption_vector.reindex(label_vector.index)], axis=1)
combine_vector

,abdomen,accessory,adaptation,afro,airsoft,akita,album,alcoholic,ankle,aqua,architecture,area,arm,army,arranging,art,athletic,atmosphere,azure,baby,bar,barechested,barefoot,bathing,bathroom,bathtub,beach,beanie,beauty,bed,bedding,beige,beverage,bicycle,bikini,black,blazer,blond,blouse,blue,...,vibes,vibrant,view,vincent,vinyl,virgin,visual,volumes,waistband,wardrobe,warhol,warholfoundation,waste,water,wear,welcome,wide,winter,woman,wool,work,working,wrap,z1975,zara,zaraaarchives,zaraarchive,zaraarchives,zaracampaign,zaracandles,zaracities,zaraedit,zaraedited,zarakids,zaraman,zaramini,zaranewin,zaraviews,zarawoman,zimmermann
0,0.0,0.320067,0.0,0.0,0.0,0.0,0.354441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.402054,0.0,0.0,0.0,0.0,0.495759,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266584,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.458611,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.136595,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.412862,0.0,0.0,0.0,0.0,0.000000,0.452418,0.0,0.000000,0.0,0.000000,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.261546,0.357266,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.111345,0.0,0.0,0.189108,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.056325,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209229,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.321754,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.178759,0.0,0.0,0.000000,0.0,0.0,0.0,0.114582,0.0,0.0,0.194605,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.057962,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.308296,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.204864,0.0,0.0,0.0,0.126281,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.117512,0.0,0.000000,0.0,0.000000,0.0
209,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.214419,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.105024,0.0
210,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
combine_logr = LogisticRegression(random_state=0).fit(combine_vector,y)
combine_pred = combine_logr.predict(combine_vector)
combine_score = combine_logr.score(combine_vector,y)
combine_score

0.9436619718309859

In [ ]:
combine_matrix = confusion_matrix(combine_pred,y)
print(combine_matrix)


[[102   7]
 [  5  99]]


We can see that the image labels and caption text together give a model accuracy of about 94%, with an approximately equal number of false positives and false negatives. 

This shows that the post captions and image labels can be used in tandem for a greater predictive accuracy than either of them alone. We can conclude that there is often important information contained within the image that is not included in the caption. 

## Task C

Creating 4 quartiles of engagement based on engagement score created in Task A. 


In [ ]:
insta_data['Engagement Quartile'] = pd.qcut(insta_data['Engagement Score'], 4, labels=['Low', 'Medium', 'High', 'Very high'])


In [ ]:
tokenized = []
for i in insta_data.Labels:
  t = i.replace('-', '')
  t = tokenize(t)
  tokenized.append(t)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized)
corpus = [dictionary.doc2bow(i) for i in tokenized]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.insta_labels')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Running topic modeling on the image labels to create 3 topic bins for the images. We decided on 3 topics because topics could be more clearly discerned than with 4 topics. 

(We used gensim for this process to return a sorted list of topic elements and weights)


In [ ]:
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, random_state = 44)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)
  

(0, '0.073*"hair" + 0.052*"beauty" + 0.043*"leg" + 0.031*"skin" + 0.031*"hairstyle" + 0.027*"human" + 0.027*"shoulder" + 0.022*"long"')
(1, '0.080*"photography" + 0.031*"photograph" + 0.029*"blackandwhite" + 0.027*"white" + 0.025*"sitting" + 0.019*"monochrome" + 0.019*"snapshot" + 0.018*"cool"')
(2, '0.098*"fashion" + 0.071*"clothing" + 0.058*"outerwear" + 0.033*"dress" + 0.033*"shoulder" + 0.027*"white" + 0.026*"standing" + 0.021*"sleeve"')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


We can see that our three topics, 0,1, and 2, each have a distinct set of words that describe it. Topic 0, or 'Close-Ups' is defined by hair, beauty, skin, and human. Topic 1, or 'Black and White', is defined by black and white photography. Topic 2, or 'Clothing and Fashion' is defined by fashion, clothing, outerwear, dress, etc. 

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
    
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
topic_zero = []
topic_one = []
topic_two = []
for i in insta_data.Labels:
  new_doc = i
  new_doc = prepare_text_for_lda(new_doc)
  new_doc_bow = dictionary.doc2bow(new_doc)
  topic_zero.append(ldamodel.get_document_topics(new_doc_bow)[0][1])
  topic_one.append(ldamodel.get_document_topics(new_doc_bow)[1][1])
  topic_two.append(ldamodel.get_document_topics(new_doc_bow)[2][1])

ldamodel.get_document_topics(new_doc_bow)


[(0, 0.03478351), (1, 0.035646748), (2, 0.9295697)]

We can see above that 93% of Zara's Instagram posts have Topic 2, 'Clothing and Fashion' as their dominant topic. 

In [ ]:

insta_data['close_ups'] = topic_zero
insta_data['black_and_white'] = topic_one
insta_data['clothing_fashion'] = topic_two

In our dataframe we can now see topic weights for each image. 

In [ ]:
topic_weights = insta_data[['Caption','close_ups','black_and_white','clothing_fashion']]

topic_weights


,Caption,close_ups,black_and_white,clothing_fashion
0,FW20 campaign. Coming soon #zaracampaign #zara...,0.189326,0.035313,0.775360
1,FW20 campaign. Coming soon #zaracampaign #zara...,0.034372,0.074030,0.891589
2,FW20 campaign. Coming soon #zaracampaign #zara...,0.055853,0.058206,0.885945
3,Archive Collection 1996/2012 — Launching a re-...,0.037284,0.038193,0.924508
4,Archive Collection 1996/2012 — Launching a re-...,0.887224,0.056717,0.056059
...,...,...,...,...
208,Zara Man and the Estate of Eduardo Chillida co...,0.038814,0.921954,0.039276
209,AWAKENING | Fine Knitwear collection. Key tran...,0.043534,0.408631,0.547831
210,AWAKENING | Fine Knitwear collection. Key tran...,0.038236,0.040452,0.921353
211,Boots evolve into modern looks with heavy lug ...,0.338620,0.218335,0.443009


In [ ]:
topic_weights.to_csv('topic_weights.csv')

Now we can compare the proportions of each topic among the highest and lowest engagement quartiles. 

In [ ]:
comparison_table = insta_data.groupby(['Engagement Quartile'])[['close_ups', 'black_and_white', 'clothing_fashion']].mean()

comparison_table = comparison_table.drop(['Medium','High'], axis=0)

comparison_table

,close_ups,black_and_white,clothing_fashion
Engagement Quartile,,,
Low,0.222539,0.330311,0.447152
Very high,0.311425,0.209586,0.478921


## Task D


From our results above, it is clear that 'Clothing and Fashion' is a consistent element of both the low and very high engagement quartiles. 

However, we see more striking differences in the other two topics categories. The very high engagement posts contain the topic 'Close-Up' at a rate almost 10% higher than the low engagement posts, and cointain the topic 'Black and White' at a rate 12% lower than the low engagement posts. 

From this, it should be clear to Zara that photos that are focused on human close-ups (i.e. faces, close human photography) and are not black and white tend to more frequently achieve higher engagements. 

While it is important to have a diversity of content, we recommend that Zara more frequently include human close-ups and less frequently include black and white photos in their Instagram posts to improve engagement. 
